In [10]:
import numpy as np
import pandas as pd
import xarray as xr
import zarr
import math
import glob
import pickle
import statistics
import scipy.stats as stats
from sklearn.neighbors import KernelDensity
import dask
import seaborn as sns
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

In [11]:
results = pickle.load(open( "../HIGH/HIGH_OUT/SASH_track_2D.p", "rb" ) )

In [13]:
weights = np.cos(np.deg2rad(results['NOAA']['DJF'].lat)) #area weighted

In [14]:
scores=[]
for reference in results:
    for index in results:
        MAE={}
        for season in ['DJF','MAM','JJA','SON']:
            ref = results[reference][season]
            x = results[index][season]
            MAE[season] = (np.abs(ref - x)).weighted(weights).sum(('lat','lon'))
        scores.append([reference, index,np.mean(MAE['DJF'].values + MAE['MAM'].values + MAE['JJA'].values + MAE['SON'].values)])

In [19]:
models = list(results.keys())

In [20]:
df = pd.DataFrame(models,columns = ['models'])
start = 0
end = len(models)
for index in models:
    df[index] = np.array(scores[start:end])[:,-1].astype(float)
    start = start + len(models)
    end = end + len(models)

In [21]:
df = df.set_index('models')

In [22]:
df.to_csv('High_2D_Linkage.csv')